In [1]:
from catboost import CatBoostClassifier

In [28]:
#import necessary dependecies
import plotly.express as px
# import seaborn as sns
# from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.cluster import KMeans
# from sklearn.preprocessing import LabelEncoder
# from sklearn.pipeline import Pipeline

# import math
# import os
import warnings
import numpy as np  
import pandas as pd, os, gc
# import matplotlib.pyplot as plt

from sklearn.metrics import roc_curve, auc, log_loss
# from sklearn.feature_selection import mutual_info_classif
# from sklearn.preprocessing import StandardScaler, RobustScaler,MinMaxScaler

%matplotlib inline
warnings.filterwarnings('ignore')
from typing import List

In [29]:
#Data Path
data_path ='data/'
# train =pd.read_csv(data_path + trains, parse_dates=['join_date'])
# test =pd.read_csv(data_path + tests, parse_dates=['join_date'])
# submission =pd.read_csv(data_path + submissions)



In [56]:
#define some fxn to read the data set
def read_train():
    train =pd.read_csv(data_path + 'train.csv', parse_dates=['join_date'])
    return train

def read_test():
    test =pd.read_csv(data_path + 'test.csv', parse_dates=['join_date'])
    return test


def read_submission():
    submission =pd.read_csv(data_path + 'SampleSubmission.csv')
    return submission

#Assign defined fxns to variables
train = read_train();test=read_test();submission=read_submission()

In [57]:
#print all dataset shape
print(train.shape, test.shape,submission.shape)

(29132, 29) (10000, 29) (210000, 2)


In [58]:
#Store Products ID in Multidimensional array
product_col = ['P5DA', 'RIBP', '8NN1',
       '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
       'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3']

products = train[train.columns[8:]]
products_t =test[test.columns[8:]]

In [59]:
products.shape

(29132, 21)

In [60]:
# k =pd.concat(objs=[train[train.columns[8:-1] ==1]], axis=1)
k = train[train.columns[8:]]
out = []
for i in range (len(k)):
    s = k.iloc[i]        
    a = s.index[s == 1]
#     out.append( ''.join(map(str,[k.columns.get_loc(col) for col in a])))
    out.append(sum([k.columns.get_loc(col) for col in a]))

In [35]:
# k =pd.concat(objs=[train[train.columns[8:-1] ==1]], axis=1)
j= test[test.columns[8:]]
out_2 = []
for i in range (len(j)):
    s = j.iloc[i]        
    a = s.index[s == 1]
#     out.append( ''.join(map(str,[k.columns.get_loc(col) for col in a])))
    out_2.append(sum([j.columns.get_loc(col) for col in a]))

In [61]:
train['impact _of_previous_subscription'] =  train['P5DA'] + train['RIBP'] +\
train['7POT']+train['66FJ']+train['GYSR']+train['SOP4']+\
train['RVSZ']+train['PYUQ']+train['LJR9']+train['N2MW']+\
train['AHXO']+train['BSTQ']+train['FM3X']+train['K6QO']+\
train['QBOL']+train['JWFN']+train['JZ9D']+train['J9JW']+\
train['GHYX']+train['ECY3']+train['8NN1']
train['impact _of_previous_subscription'] = train['impact _of_previous_subscription'].astype(int)

In [62]:
test['impact _of_previous_subscription'] = test['P5DA'] + test['RIBP'] +\
test['7POT']+test['66FJ']+test['GYSR']+test['SOP4']+\
test['RVSZ']+test['PYUQ']+test['LJR9']+test['N2MW']+\
test['AHXO']+test['BSTQ']+test['FM3X']+test['K6QO']+\
test['QBOL']+test['JWFN']+test['JZ9D']+test['J9JW']+\
test['GHYX']+test['ECY3']+test['8NN1']
test['impact _of_previous_subscription'] = test['impact _of_previous_subscription'].astype(int)
test['impact _of_previous_subscription'] 

0       1
1       2
2       2
3       1
4       1
       ..
9995    2
9996    1
9997    1
9998    1
9999    1
Name: impact _of_previous_subscription, Length: 10000, dtype: int32

In [63]:
train=train.fillna(train.mode().iloc[0])
test=test.fillna(test.mode().iloc[0])

In [64]:
cols = ['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
       'occupation_code','impact _of_previous_subscription','occupation_category_code',
         'P5DA', 'RIBP', '8NN1','7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 
         'N2MW', 'AHXO','BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3']

In [65]:
train = train[cols]
test = test[cols]


In [66]:
#Axis Transformation
train = train.melt(id_vars=train.columns[:9], value_vars=products, var_name = "PCODE", value_name="Label" )
test = test.melt(id_vars=test.columns[:9], value_vars=products, var_name = "PCODE", value_name="Label" )

In [67]:
train.head()

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,impact _of_previous_subscription,occupation_category_code,PCODE,Label
0,4WKQSBB,2019-01-02,F,M,1987,1X1H,2A7I,2,T4MS,P5DA,0
1,CP5S02H,2019-01-06,F,M,1981,UAOD,2A7I,2,T4MS,P5DA,0
2,2YKDILJ,2013-01-06,M,U,1991,748L,QZYX,3,90QI,P5DA,0
3,2S9E81J,2019-01-08,M,M,1990,1X1H,BP09,2,56SI,P5DA,0
4,BHDYVFT,2019-01-08,M,M,1990,748L,NO3L,2,T4MS,P5DA,0


In [68]:
#A merger
train['combiner']='x'

In [69]:
#Set
data=pd.concat([train,test],sort=False).reset_index(drop=True)

In [70]:
#preview random 3 sample
data.isnull().any()

ID                                  False
join_date                           False
sex                                 False
marital_status                      False
birth_year                          False
branch_code                         False
occupation_code                     False
impact _of_previous_subscription    False
occupation_category_code            False
PCODE                               False
Label                               False
combiner                             True
dtype: bool

In [71]:
#Confirm Distribution (TEST AND SAMPLE SUBMISSION FILE)
print('==' * 18);print('TRAIN LABEL DISTRIBUTION');print('==' * 18);print(train['Label'].value_counts())
print('==' * 18);print('TEST LABEL DISTRIBUTION');print('==' * 18);print(test['Label'].value_counts())
print('==' * 18);print('SAMPLE SUBMISSION LABEL DISTRIBUTION');print('==' * 18);print(submission['Label'].value_counts())

TRAIN LABEL DISTRIBUTION
0    545419
1     66353
Name: Label, dtype: int64
TEST LABEL DISTRIBUTION
0    197147
1     12853
Name: Label, dtype: int64
SAMPLE SUBMISSION LABEL DISTRIBUTION
0    197147
1     12853
Name: Label, dtype: int64


In [72]:
test_label = test
train=data[data.combiner.notnull()].reset_index(drop=True);test=data[data.combiner.isna()].reset_index(drop=True)
train.drop('combiner', inplace=True, axis=1);test.drop(['Label','combiner'], inplace=True, axis=1)
print(train.shape, test.shape, submission.shape)

(611772, 11) (210000, 10) (210000, 2)


In [73]:
#Concatenate new column towards submission requirement
train['ID X PCODE'] = train['ID'] + ' X ' + train['PCODE'];test['ID X PCODE'] = test['ID'] + ' X ' + test['PCODE']

In [74]:
#Re-arrange column (not necessary)
train=train[['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
       'occupation_code','impact _of_previous_subscription', 'occupation_category_code', 'PCODE','ID X PCODE',  'Label']]
test=test[['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
       'occupation_code','impact _of_previous_subscription', 'occupation_category_code', 'PCODE','ID X PCODE']]


In [75]:
#Lets Check for data co-existence in Train/Test
for col in ['PCODE','occupation_code', 'branch_code','occupation_category_code']:
    train_col = train[col].unique();test_col = test[col].unique()
    print ('Not in Test ' + col + ' ',[i  for i in train_col if i not in test_col])
    print ('Not in Train ' + col + ' ',[i  for i in test_col  if i not in train_col]);print('==' * 30)

Not in Test PCODE  []
Not in Train PCODE  []
Not in Test occupation_code  ['IE90', 'Q0LY', 'JSAX', '6XXU', '8HRZ', 'OQMY', 'INEJ', 'VZN9', 'UC7E', 'PSUY', 'WSRG', 'JQH3', 'LGTN', '738L', 'QQUP', '2XZ1', 'CAAV', 'LLLH', 'W1X2', 'DHSN', 'IX8T', '2US6', 'ZWPL', 'MEFQ', '9B5B', 'JUIP', 'BFD1', 'A4ZC', 'IMHI', 'E5PF', 'GZA8', '3YQ1', 'PJR4', 'NDL9', 'PPNK', '2686', '5LNN', '374O', 'URYD', 'M0WG', 'KBWO', 'ONY7', 'VYSA', 'KUPK', 'R7GL', 'HSVE', 'BER4', '6SKY', 'RH2K', 'ZHC2', 'W3ZV', 'FLXH', 'UYDZ', 'YJXM', '59QM']
Not in Train occupation_code  ['0ZND', '8CHJ', '9F96', 'HSI5', '93OJ', 'BIA0', 'E2MJ', 'JBJP', '0FOI']
Not in Test branch_code  []
Not in Train branch_code  []
Not in Test occupation_category_code  []
Not in Train occupation_category_code  []


In [76]:
options = ['0ZND', '8CHJ', '9F96', 'HSI5', '93OJ', 'BIA0', 'E2MJ', 'JBJP', '0FOI']

len(test[test['occupation_code'].isin(options) ])
test_opt = test[~(test['occupation_code'].isin(options)) ]

In [77]:
train.isnull().any().sum()

0

In [78]:
test.isnull().any().sum()

0

In [79]:
# # LABEL ENCODE
# def encode_LE(train,test,cols,verbose=True):
#     for col in cols:
#         df_comb = pd.concat([train[col],test[col]],axis=0)
#         df_comb,_ = df_comb.factorize(sort=True)
#         nm = col
#         if df_comb.max()>32000: 
#             train[nm] = df_comb[:len(train)].astype('int32')
#             test[nm] = df_comb[len(train):].astype('int32')
#         else:
#             train[nm] = df_comb[:len(train)].astype('int16')
#             test[nm] = df_comb[len(train):].astype('int16')
#         del df_comb; x=gc.collect()
#         if verbose: print(nm,', ',end='')

In [55]:
encode_LE(train, test,['ID','branch_code', 'occupation_code','occupation_category_code','PCODE','sex','marital_status'])

ID , branch_code , occupation_code , occupation_category_code , PCODE , sex , marital_status , 

In [80]:
import datetime as dt
train['day'] = train['join_date'].dt.day
train['month'] = train['join_date'].dt.month
train['year'] = train['join_date'].dt.year
train.drop('join_date', axis=1, inplace=True)


In [81]:
test.head()
testk = test

In [82]:
test['day'] = test['join_date'].dt.day
test['month'] = test['join_date'].dt.month
test['year'] = test['join_date'].dt.year
test.drop('join_date', axis=1, inplace=True)


test_opt['day'] = test_opt['join_date'].dt.day
test_opt['month'] = test_opt['join_date'].dt.month
test_opt['year'] = test_opt['join_date'].dt.year
test_opt.drop('join_date', axis=1, inplace=True)


In [83]:
test.head()

,ID,sex,marital_status,birth_year,branch_code,occupation_code,impact _of_previous_subscription,occupation_category_code,PCODE,ID X PCODE,day,month,year
0,F86J5PC,M,M,1984,94KC,DZRV,1,90QI,P5DA,F86J5PC X P5DA,12,1,2018
1,H6141K3,M,M,1996,1X1H,J9SY,2,90QI,P5DA,H6141K3 X P5DA,10,1,2019
2,RBAYUXZ,F,W,1968,UAOD,2A7I,2,T4MS,P5DA,RBAYUXZ X P5DA,1,1,2020
3,KCBILBQ,M,M,1989,94KC,2A7I,1,T4MS,P5DA,KCBILBQ X P5DA,2,1,2019
4,LSEC1ZJ,F,M,1982,UAOD,0KID,1,T4MS,P5DA,LSEC1ZJ X P5DA,2,1,2020


# TRAIN AND TEST FILE PREPARED

In [84]:
t = test['ID X PCODE']
t

0         F86J5PC X P5DA
1         H6141K3 X P5DA
2         RBAYUXZ X P5DA
3         KCBILBQ X P5DA
4         LSEC1ZJ X P5DA
               ...      
209995    0GMU5UH X ECY3
209996    I9W11CD X ECY3
209997    42WTEGT X ECY3
209998    8EKC4O9 X ECY3
209999    WHII93K X ECY3
Name: ID X PCODE, Length: 210000, dtype: object

In [85]:
drop_col = ['ID','ID X PCODE']
outlier_col = data.select_dtypes(include=['int64', 'float64']).columns #non catagorical columns


In [86]:
train_new = train
test_new = test
train.drop(drop_col,axis=1,inplace= True)
test.drop(drop_col,axis=1,inplace= True)

In [313]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler

def scaler(data, scaler= MinMaxScaler()): # not implmented in optimized way yet
        data[:] = scaler.fit_transform(data[:]) #scaleing the given data
#         self.data = data
        print("\n---- Scaling the data based on",scaler)
        return data
    
train_n = scaler(train)



---- Scaling the data based on MinMaxScaler()


In [314]:
test_n =scaler(test)


---- Scaling the data based on MinMaxScaler()


In [87]:
from sklearn import preprocessing 
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier

import scipy.stats as stat
from sklearn import svm

from sklearn.metrics import *
import datetime


k_fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

In [88]:
def get_eff_model(y_test,y_pred):
        
            metrics = pd.DataFrame()
#             metrics["MSE"] = mean_squared_error(self.y_test,self.y_pred)
            metrics["Loss"] = log_loss(y_test,y_pred)
#             metrics["Score"] = -(r2_score(y_test,y_pred))
#             metrics["Kappa"] = cohen_kappa_score(self.y_test, self.y_pred)
#             metrics["ROC_Auc"] = roc_auc_score(y_test, y_pred)
#             metrics["precision"] = precision_score(self.y_test, self.y_pred)
#             metrics["recall"] = recall_score(self.y_test, self.y_pred)
#             metrics["f1_score"] = f1_score(self.y_test, self.y_pred)
#             metrics["accuracy"] = accuracy_score(y_test, y_pred)
        

        
            return metrics

In [89]:
def make_it_stratified(data,target,model_name):
        X = data.drop(target,axis=1).values
        y=(data[target]).values.ravel()
 
        eff_kfold =[]
        eff =[]
        model_pred = []
        kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
                       
                       
                       
        if model_name== "RandomForestClassifier":
            model = RandomForestClassifier()
        elif model_name == "LogisticRegression":
            model = LogisticRegression(solver='saga', random_state=0)
        elif model_name == "DecisionTreeClassifier":
            model = DecisionTreeClassifier()
        elif model_name == "XG_Boost":
            
            model = xgb.XGBClassifier()
        elif model_name =="Multilayer Perceptron":
            print("not implemented yet")
        elif model_name == "svm" :
            model = svm.SVC(kernel='linear', C=0.01)
        elif model_name == "adboost":
            model =AdaBoostClassifier()
        elif model_name == "gradienBoost":
            model = GradientBoostingClassifier()
                       

        #enumerate the splits and summarize the distributions
        i=0
        for train_ix, test_ix in kfold.split(X, y):
            i =i+1
            print("k_fold -{}   with {} model".format(i,model_name))

           
     
            # select rows
            X_train,X_test = X[train_ix], X[test_ix]
            y_train, y_test = y[train_ix], y[test_ix]

            model = model.fit(X_train,y_train)
            y_pred = model.predict_proba(X_test)[:,1]
            pred = model.predict(X_test)

     
            print("k_fold -{}   with {} end!".format(i,model_name))
        print("log loss",log_loss(y_test,y_pred))
        print("accuracy",accuracy_score(y_test, pred))

        return model
            
#         df_eff = pd.concat(eff)
#         df_eff =pd.DataFrame(df_eff.mean()).transpose()
#         df_eff.index=[model_name]

       


In [90]:
classifiers =["RandomForestClassifier","LogisticRegression",
              "DecisionTreeClassifier", "XG_Boost","gradienBoost","adboost"]

In [91]:
train.head()
non_cat_col = train.select_dtypes(include=['int64','int32', 'float64']).columns # non catagorical variable
cat_col = train.select_dtypes(exclude=['int64','int32', 'float64']).columns #  catagorical variable
# train.info()
cat_col

Index(['sex', 'marital_status', 'branch_code', 'occupation_code',
       'occupation_category_code', 'PCODE'],
      dtype='object')

In [92]:
from sklearn.model_selection import train_test_split

X = train.drop('Label',axis=1)
y=(train['Label'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1,stratify =y,random_state=1)#use stratified kfold distribution for imbalanced class distributin


In [93]:
train.shape

(611772, 12)

In [94]:
from catboost import CatBoostClassifier

clf = CatBoostClassifier(
    iterations=50, 
    learning_rate=0.1, 
    custom_loss=['AUC', 'Accuracy']
)


clf.fit(X_train, y_train, 
        cat_features=cat_col, 
        eval_set=(X_test, y_test), 
        verbose=False,
#         plot=True

)

# print('CatBoost model is fitted: ' + str(clf.is_fitted()))
# print('CatBoost model parameters:')
# print(clf.get_params())

In [95]:
print('CatBoost model is fitted: ' + str(clf.is_fitted()))
print('CatBoost model parameters:')
print(clf.get_params())

CatBoost model is fitted: True
CatBoost model parameters:
{'custom_loss': ['AUC', 'Accuracy'], 'learning_rate': 0.1, 'iterations': 50}


In [96]:
y_pred = clf.predict(data=X_test)
print(log_loss(y_test,y_pred))

1.4119807156673576


In [326]:
clf.g
# for classifier in classifiers:
# dmodel = make_it_stratified(train_n,'Label',"XG_Boost")



model = xgb.XGBClassifier()

model.fit(X,y)

# X = train_n.drop('Label',axis=1).values
# train_n.isnull().any()

XGBClassifier()

In [328]:
pred = model.predict_proba(test_n)[:,1]



In [ ]:
clf

In [323]:
# testk =pd.read_csv(data_path + tests, parse_dates=['join_date'])
# testk = testk.melt(id_vars=testk.columns[:8], value_vars=products, var_name = "PCODE", value_name="Label" )
# tesk testk[testk["Label"] == 1].counts()
# testk["Label"] 


In [330]:
df = pd.DataFrame(t)
df["Label"] = pred
df

,ID X PCODE,Label
0,F86J5PC X P5DA,0.019085
1,H6141K3 X P5DA,0.029368
2,RBAYUXZ X P5DA,0.025138
3,KCBILBQ X P5DA,0.019085
4,LSEC1ZJ X P5DA,0.015259
5,EK65KRH X P5DA,0.016882
6,M3N7TLU X P5DA,0.030203
7,QAC6HOX X P5DA,0.030203
8,T8B9JK0 X P5DA,0.019085
9,62M79HM X P5DA,0.016142


In [331]:
df.to_csv("Zimint-challeng-Baseline-4.csv")

In [116]:
df_new.to_csv("t.csv",index=False)

In [115]:
df_new.set_index(df_new['ID X PCODE'])

,ID X PCODE,Label
ID X PCODE,,


In [112]:
df_new


,ID X PCODE,Label


In [117]:
d = pd.read_csv("Zimint-challeng-Baseline-1.csv")

In [118]:
d.head()

,ID X PCODE,Label
0,F86J5PC X P5DA,0.011993
1,H6141K3 X P5DA,0.010886
2,RBAYUXZ X P5DA,0.010834
3,KCBILBQ X P5DA,0.011993
4,LSEC1ZJ X P5DA,0.011993


In [119]:
d.index


RangeIndex(start=0, stop=210000, step=1)